In [1]:
#from collections import deque
#import networkx as nx
import random
import numpy as np
import heapq
import math
from matplotlib import pylab
seed = 42
random.seed(seed)
np.random.seed(seed)

#Algoritmo GBFS

In [2]:
class Node:
    def __init__(self, x, y, cost):
        self.x = x
        self.y = y
        self.cost = cost
    def __lt__(self, other):
        return self.cost < other.cost

def euclidean_distance(x1, y1, x2, y2):
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

def greedy_best_first_search(grid, start, goal):
    rows = len(grid)
    cols = len(grid[0])
    pq = []
    heapq.heappush(pq, Node(start[0], start[1], 0))
    visited = set()
    visited.add((start[0], start[1]))
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    while pq:
        current = heapq.heappop(pq)
        if (current.x, current.y) == goal:
            print(f"Goal reached at ({current.x}, {current.y})")
            return
        for d in directions:
            new_x, new_y = current.x + d[0], current.y + d[1]
            if 0 <= new_x < rows and 0 <= new_y < cols and grid[new_x][new_y] == 0 and (new_x, new_y) not in visited:
                cost = euclidean_distance(new_x, new_y, goal[0], goal[1])
                heapq.heappush(pq, Node(new_x, new_y, cost))
                visited.add((new_x, new_y))

    print("Goal not reachable")

grid = np.array([
    #[0, 1, 1, 1],
    [0, 0, 1, 1],
    [1, 0, 1, 1],
    [1, 0, 0, 1],
    [1, 1, 0, 0]
])

start = (0, 0)
goal = (3, 3)
greedy_best_first_search(grid, start, goal)

Goal reached at (3, 3)


# Algoritmo A*

In [3]:
class Node:
    def __init__(self, position, parent=None):
        self.position = position
        self.parent = parent
        self.g = 0  # Distance from start node
        self.h = 0  # Heuristic to goal
        self.f = 0  # Total cost

    def __eq__(self, other):
        return self.position == other.position

    def __lt__(self, other):
        return self.f < other.f

    def __repr__(self):
        return f"({self.position}, f: {self.f})"

def heuristic(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def astar(maze, start, end):
    open_list = []
    closed_list = set()
    start_node = Node(start)
    end_node = Node(end)
    heapq.heappush(open_list, start_node)

    while open_list:

        current_node = heapq.heappop(open_list)

        closed_list.add(current_node.position)

        if current_node == end_node:
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]

        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]:
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze)-1]) - 1) or node_position[1] < 0:
                continue

            if maze[node_position[0]][node_position[1]] != 0:
                continue

            new_node = Node(node_position, current_node)

            if new_node.position in closed_list:
                continue

            new_node.g = current_node.g + 1
            new_node.h = heuristic(new_node.position, end_node.position)
            new_node.f = new_node.g + new_node.h
            if add_to_open(open_list, new_node):
                heapq.heappush(open_list, new_node)

    return None

def add_to_open(open_list, neighbor):
    for node in open_list:
        if neighbor == node and neighbor.g > node.g:
            return False
    return True

maze = np.array([
    [0, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0]
])

start = (0, 0)
end = (4, 6)
path = astar(maze, start, end)
print(path)

[(0, 0), (1, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 6), (4, 6)]
